## Descripcion del problema 
El trabajo consiste en desarrollar un sistema en Python capaz de recuperar, procesar y organizar información proveniente de 
distintas fuentes de la Web, aplicando varias técnicas de obtención de datos: 
consumo de APIs, web scraping y lectura de feeds RSS.

El programa debe obtener información de tres tipos de fuentes diferentes:

APIs: OpenAlex y The Lens, para consultar artículos científicos y patentes.

Sitios web: Eventseye, Nferias y 10Times, para extraer datos de eventos y ferias mediante scraping.

Feeds RSS: fuentes relacionadas a comercio internacional (WTO y Comtrade Plus) para obtener noticias recientes.

Cada conjunto de datos debe ser procesado y almacenado en archivos CSV, asegurando consistencia en los metadatos extraídos. Además, el sistema debe 
incluir una interfaz por consola que permita ejecutar las consultas y visualizar los datos generados.

## Análisis y Diseño

### Parte 1: APIs
El diseño implementado esta creado con distintas funciones para dividir las responsabilidades.
### Metodo para obtener los datos de la API Open Alex:
    -Funcion: buscar_articulos_openalex()
    -Proposito: Este se encarga de obtener los articulos mas recientes, siendo el unico metodo de contacto con la API.
    -Diseño: 
        -Se utilizo la biblioteca "requests" para realizar las solicitudes HTTP.
        -En la cabecera del metodo se ingresan los paramaetros para realizar la consulta (por-page y search), permitiendo que la busqueda sea flexible.
        -Incluye manejo de errores.

### Metodos para procesamiento y extracion de Metadatos:
    -Funciones: extraer_datos_de_autores_intitucion(), extraer_palabras_clave(), extraer_pais_de_publicacion(), extraer_campos_de_estudio() y extraer_resumen().
    -Proposito: Este conjunto de funciones permite la division de distintas responsabilidades, esto facilita la busqueda de errores, en caso de que este fallando un caso en especial.
    -Diseño:
        -Cada metodo se encarga de extraer un tipo de metadato.
        -Inclute Manejo de errores.

### Metodo de transformacion:
    -Funcion: extraer_metadatos()
    -Proposito: Se encarga de convertir la lista de articulos obtenida, en una lista de diccionarios para luego ser guardada en un archivo CSV.
    -Diseño:
        -Itera sobre cada articulo encontrado, en cada uno de estos extrae los metadatos utilizando los metodos creados, limpia los datos procesados, luego los ensambla en un diccionario y este lo agrega a una lista.

### Metodo de persistencia de datos:
    -Funcion: guardar_en_csv()
    -Proposito: Toma los datos limpios de cada articulo almacenado y los guarda en ..data/articulos.CSV
    -Diseño:
        -Se utilizo la biblioteca csv, nos brinda los metodos utiles como DictWriter permitiendonos escribir las cabeceras y las filas.

### Complejidad de metodos principales:
    -buscar_articulos_openalex(): Su complejidad es de O(n) ya que n seria la cantidad de articulos que OpenAlex devuelve en results, la complejidad estaria ligada al tamaño de la respuesta.
    -extraer_metadatos(): Su complejidad es de O(n) al tener un for que va a recorrer segun la cantidad de articulos obtenidos.
    -guardar_en_csv(): Su complejidad es de O(n) al escribir  n filas en el archivo CSV, cada fila implica una operacion.
    -mostrar_datos_csv_openalex(): Su complejidad es de O(n) al leer n filas del archivo CSV y las imprime.
    -consultar_openalex(): Su complejidad es de O(n) al ejecutar las siguientes funciones, buscar_articulos_openalex(), extraer_metadatos(), guardar_en_csv()

### Estructura de datos aplicadas:
    -Diccionarios:
        -Permite representar objetos con propiedas.
        -El metodo extraer_metadatos() convierte los articulos a dicconarios, esto permite un acceso rapido a cada metadato del articulo.
        -El metodo buscar_articulos_openalex() almacena los paramaetros de consulta en un diccionario, ya que la biblioteca requests espera los parametros en la URL.
    -Listas:
        -Contenedores que representa colecciones ordenadas
        -El metodo extraer_metadatos() utiliza un contenedor que es una lista de diccionarios.
    -Conjuntos:
        -Este no permite datos duplicados, garantiza la unicidad de datos.
        -El metodo extraer_pais_de_publicacion utiliza set ya que un articulo puede tener varios autores del mismo pais o institucion, evitando duplicarlos.


### Parte 2: Web Scraping

### Librerias utilizadas:
    -RobotFileParser: Usado para poder realizar las consultas de robots.txt y poder respetar las paginas a scrapear
    -request y BeautifulSoup: La base del scrap
    -urljoin: Utilizado para poner unir URL's más fácilmente
    -re: Usado para obtener el correo en determinadas páginas, ya que cambiaba de posición constantemente
    -os y csv: Utilizado para poder crear y modificar los archivos .csv
    -cloudscraper: Utilizado exclusivamente para acceder 10Times, ya que usando requests denegaba el acceso.


### - Análisis de los sitios web objetivo
El proceso de scraping en nferias y eventseye fueron parecidos. Ambos tienen una estructura simple y parecida. Nferias requiere un poco más técnica por su forma de organizar la información. Ambos poseian una estructura HTML similar, intuitiva y lógica.
En cambio, 10times si requirió una técnica mucho más avanzada, debido a la utilización de JavaScript para cargar contenido dinámico. Además de tener una estructura HTML insual y confusa.

 Complejidad espacial y temporal:
 
    NFERIAS: 
    Complejidad temporal: O(i. k. j) siendo I: limiteIndustrias, k: ferias_por_industria y j: la cantidad de paginas que se vayan a requerir para llegar a k ferias

    Complejidad espacial: O(1)

    EVENTSEYE:
        Complejidad temporal: O(L · S · E) siendo L: letras, S: sectores y E: eventos

        Complejidad espacial: O(1)

    10TIMES:
        Complejidad temporal: O(C · E) siendo C: categorias y E: eventos

        Complejidad espacial: O(1)

### - Diseño del scraper
 El diseño del scraper de nferias y eventseye son similares, parten de una URL semillas, realiza las solicitudes para poder seguir extrayendo URL's hasta llegar a la URL donde extraerá la informacion que nos interesa.
 
 El diseño de 10times no es muy diferente a los demas. Parte de una url semilla en la que recorre las categorias y  en cada categoria los eventos, por cada evento visitado se extrae su informacion. Cuando los eventos alcanzan el limite preestablecido el scrap, este entra en una nueva categoria. Cuando esta alcanza el limite termina la ejecucion. Este algoritmo no cuenta con paginacion, no extrae la web del evento ni el contacto, ya que la pagina carga sus paginas dinamicamente con JavaScript y usa funciones del mismo lenguaje para acceder a los mencionados datos, problema que no hemos podido resolver.
 Otro problema con el que cuenta este algoritmo, es que no puede usarse tan seguido. Una vez usado se debe esperar unos minutos para volverse a usar, ya que al querer hacerlo empieza a tener errores que antes no tenia.

### - Estrategias de extracción de datos
    El objetivo principal de extraccion a cada página consistia en llegar a cada evento/feria individual en la menor cantidad de request posibles. Por lo que optamos por empezar las request desde una pagina que ya contenga todas las industrias/sectores de la pagina. Una vez llegado al evento la extraccion de toda la informacion necesario, la extracción pasa un proceso antes de guardarse:
    Se tokeniza, se saca espacios extras y une todo con solamente un espacio, si es necesario. Si la información es una estructura de datos que no sea una cadena de texto, se la transforma en una. En caso de tener texto o signos de puntuacion no deseados, se utiliza regex para extraerlos.


### - Estructuras de datos aplicadas
Además de listas, se utilizaron diccionarios para almanenar los datos internamente y la generación de un archivo .csv para la lectura del usuario.


#### Parte 3: Analisis RSS
##### 1. Introducción

    Se desarrolló un módulo encargado de consumir, procesar y almacenar información proveniente de feeds RSS utilizando la librería feedparser. El sistema permite consultar  BBC o El País, extraer datos relevantes de cada noticia, identificar países mencionados y registrar la información de manera persistente evitando duplicados.

    El procesamiento principal se encuentra implementado en el archivo:
    /src/rss/lector.py

##### 2. Arquitectura General del Parser

    El parser fue diseñado con tres objetivos principales:

    Consumir varios feeds RSS de forma dinámica según la selección del usuario.

    Normalizar y estructurar la información en un formato interno uniforme.

    Persistir los datos sin duplicar noticias, aun si el usuario ejecuta el lector varias veces.

    La función principal del módulo es leer_rss(url, nombre_pagina).

##### 3. Lectura de RSS con feedparser

    Se empleó la librería feedparser, que permite analizar documentos RSS mediante el método:

    feed = feedparser.parse(url)


    Cada item del feed se representa como entry y contiene atributos tales como:(cada entry se obtienen de feed.entries al recorrer con for)

    entry.title

    entry.summary

    entry.published

    entry.tags (si existen)

    A partir de estos datos se construye un diccionario por noticia con la información relevante.

##### 4. Extracción de Información Relevante
4.1 Campos almacenados por noticia

    Para cada entrada del RSS se extraen y procesan los siguientes datos:

    Título

    Descripción

    Fecha de publicación (con traducción del día al español)

    Países relacionados

    ID única generada por hash

##### 5. Complejidad temporal y espacial de cada funcion

### Funcion: obtener_dia_semana(fecha_str):
Se realiza un split, y operaciones sobre cada string:
Temporal y espacial: O(n)

### Funcion: buscar_pais_en_data(arreglo, titulo, descripcion)
Temporal:
    Recorre:
        1.Set paises: O(n)
        2.Tags: O(n)
        3. Buscar titulo y desc: O(n)
Espacial:
    Set con n paises: O(n)

### Funcion: obtener_id_visitadas()
Temporal:
    Recorre:
        1. Archivo en memoria: O(n)
Espacial:
    Set con n lineas: O(n)


### Funcion guardar_id_visitada(id_noticia): 
Temporal y espacial:
    1. Escribe sobre un archivo: O(1)

### Funcion leer_rss(url,nombre_pagina):
Temporal:
    1. Recorre feed entries: O(n)

Espacial: 
    1.Las guarda en una lista con n elementos: O(n)

###  guardar_csv(noticias, nombre_pagina):
Temporal:
    1. Recorre nuevos registros y existentes: O(n+m) = O(n)
Espacial:
    1. Crea una lista combinada de longitud total n

### Visualizar csv rss:
Temporal.
    1. Imprimir y leer cada fila: O(n)
Espacial: O(1)

### Mostrar noticias:
Temporal:
    1. Recorre la lista de noticias: O(n)
Espacial: O(1)
